Задание 7. Транспортная задача

Дана транспортная задача: A1, A2, A3 – поставщики с запасами a1, a2, a3 однородного груза,

B1, B2, B3, B4, B5 – потребители с потребностями b1, b2, b3, b4, b5

Матрица тарифов C = (c_ij), где i = 1, 2, 3 и j = 1, 2, 3, 4, 5, содержит стоимости перевозки единицы груза
из пункта Ai в пункт Bj

Требуется найти минимальный по стоимости план перевозки груза от поставщиков
к потребителям такой, чтобы был вывезен весь груз и все потребности были удовлетворены.<br>

1. Убедитесь, что транспортная задача закрытого типа

2. Найдите какой-либо допустимый план перевозки груза (можно использовать методы северо-западного угла, минимальной стоимости или Фогеля)

3. Найдите оптимальный план перевозки груза (план минимальной стоимости) методом потенциалов

4. Проверьте результаты вычислений на компьютере

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import Bounds, linprog
np.set_printoptions(suppress=True)

In [2]:
C = np.array([list(map(int, input(f'Введите {i+1} строку матрицы C (числа через пробел):\n').split())) for i in range(3)])
print('\nМатрица C:')
print(C)


Матрица C:
[[10 18  8 10  4]
 [15 11 12 13  5]
 [ 6  9 16 17 21]]


In [3]:
a1, a2, a3 = map(int, input('Введите через пробел числа a1, a2, a3:\n').split())

In [4]:
A = np.array([a1, a2, a3])

In [6]:
b1, b2, b3, b4, b5 = map(int, input('Введите через пробел числа b1, b2, b3, b4, b5:\n').split())

In [7]:
B = np.array([b1, b2, b3, b4, b5])

In [8]:
N = len(A)
M = len(B)

In [9]:
df = pd.DataFrame(C, columns=B, index=A)
df.columns.name = 'aᵢ \\ bⱼ'
df

aᵢ \ bⱼ,130,150,210,270,150
250,10,18,8,10,4
390,15,11,12,13,5
270,6,9,16,17,21


In [10]:
if sum(A) == sum(B):
    print('Задача закрытого типа')
else:
    print('Задача открытого типа')

Задача закрытого типа


Допустимый план перевозки груза методом минимальной стоимости

In [14]:
P = np.zeros_like(C)
INF = 1_000_000_000
C_ = C.copy()
A_ = A.copy()
B_ = B.copy()
i = 0
j = 0
while not np.all(C_ == INF):
    i, j = np.unravel_index(C_.argmin(), C_.shape)
    P[i][j] = min(A_[i], B_[j])
    if A_[i] <= B_[j]:
        C_[i].fill(INF)
        B_[j] -= A_[i]
        A_[i] = 0
    else:
        C_[:, j].fill(INF)
        A_[i] -= B[j]
        B_[j] = 0    
if np.count_nonzero(P) == N + M - 1:
    print('Опорный план является невырожденным')
else:
    print('Опорный план является вырожденным')

F = 0
for i in range(N):
    for j in range(M):
        F += C[i][j] * P[i][j]
print('Полученный план перевозки груза:\n', P)
print('Значение целевой функции для этого опорного плана F =', F)

Опорный план является невырожденным
Полученный план перевозки груза:
 [[  0   0 100   0 150]
 [  0  10 110  30   0]
 [130 140   0   0   0]]
Значение целевой функции для этого опорного плана F = 5260


Оптимальный план перевозки груза

In [12]:
A_ub = np.array([[1 if i in range(j*M, M*(j+1)) else 0 for i in range(N*M)] for j in range(N)])
b_ub = A

b_eq = B
A_eq = np.array([[1 if i % M == j else 0 for i in range(N*M)] for j in range(M)])

res = linprog(C.flatten(), A_ub, b_ub, A_eq, b_eq, options={'disp': True})

X = res.x.round(3).reshape(N, M)
F_min = res.fun.round(3)

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 175.0               
0.135600791367      0.135600791367      0.135600791367      0.8751036980991  0.135600791367      1123.751111045      
0.02552643481705    0.02552643481705    0.02552643481705    0.8211271450878  0.02552643481736    3547.497414593      
0.005779230716104   0.005779230716104   0.005779230716104   0.7956920616757  0.005779230715972   6025.825253051      
0.002735724435628   0.002735724435628   0.002735724435628   0.5761419510577  0.002735724435565   6927.414102377      
0.0001984668961158  0.0001984668961236  0.0001984668961236  0.9286376492988  0.0001984668962583  7884.086056249      
1.616561686027e-08  1.616561676846e-08  1.616561640105e-08  0.9999268933877  1.616562545585e-08  7969.992825244      
8.108681551001e-13  8.10809252922e-13   8.098572317811e-1

C:\Users\irong\AppData\Local\Temp/ipykernel_30180/1981003704.py:11: OptimizeWarning: Solving system with option 'cholesky':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'cholesky' to False.
  res = linprog(C.flatten(), A_ub, b_ub, A_eq, b_eq, options={'disp': True})
C:\Users\irong\AppData\Local\Temp/ipykernel_30180/1981003704.py:11: OptimizeWarning: Solving system with option 'sym_pos':True failed. It is normal for this to happen occasionally, especially as the solution is approached. However, if you see this frequently, consider setting option 'sym_pos' to False.
  res = linprog(C.flatten(), A_ub, b_ub, A_eq, b_eq, options={'disp': True})
c:\users\irong\desktop\python\lib\site-packages\scipy\optimize\_linprog_ip.py:117: LinAlgWarning: Ill-conditioned matrix (rcond=5.56333e-18): result may not be accurate.
  return sp.linalg.solve(M, r, sym_pos=sym_pos)


In [16]:
print('Оптимальный план перевозки груза (X*):\n', X)
print('Оптимальная стоимость перевозки:\nF_min = F(X*) =', F_min)

Оптимальный план перевозки груза (X*):
 [[  0.   0. 210.  40.   0.]
 [  0.  10.   0. 230. 150.]
 [130. 140.   0.   0.   0.]]
Оптимальная стоимость перевозки:
F_min = F(X*) = 7970.0
